In [1]:
from pyspark import SparkConf, SparkContext
# SparkConf : 환경 설정
# SparkContext : 실제 개발 프로그램. 드라이브
#conf = SparkConf().setMaster("yarn") # yarn으로 하면 자동으로 하둡을 사용하게 된다
conf = SparkConf().setMaster("local")
sc = SparkContext(conf=conf) # sc 변수명 바꾸면 안된다. 실무에서는 스파크 서브밋 이라는 것을 사용한다. 그때 필수

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/22 11:11:46 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
filepath='/home/ubuntu/working/spark-examples/data/restaurant_reviews.csv'
lines = sc.textFile(f"file:///{filepath}")

In [3]:
lines.collect()

['id,item,cateogry,reviews,',
 '0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [4]:
header = lines.first()
header

'id,item,cateogry,reviews,'

In [5]:
datas = lines.filter(lambda row : row !=header)
datas

PythonRDD[3] at RDD at PythonRDD.scala:53

In [6]:
datas.collect()

['0,짜장면,중식,125,',
 '1,짬뽕,중식,235,',
 '2,김밥,분식,32,',
 '3,떡볶이,분식,534,',
 '4,라멘,일식,223,',
 '5,돈가스,일식,52,',
 '6,우동,일식,12,',
 '7,쌀국수,아시안,312,',
 '8,햄버거,패스트푸드,12,',
 '9,치킨,패스트푸드,23']

In [7]:
# transformation은 task이고 함수로 정의한다.
def parse_task(row):
    fields = row.split(',')
    
    key = fields[2]
    value = int(fields[3])
    
    return key, value

In [8]:
parse_task('7,쌀국수,아시안,312,')

('아시안', 312)

In [11]:
category_reviews_add = datas.map(parse_task)
category_reviews_add

PythonRDD[5] at RDD at PythonRDD.scala:53

In [13]:
category_reviews_add.collect()

[('중식', 125),
 ('중식', 235),
 ('분식', 32),
 ('분식', 534),
 ('일식', 223),
 ('일식', 52),
 ('일식', 12),
 ('아시안', 312),
 ('패스트푸드', 12),
 ('패스트푸드', 23)]

In [19]:
category_review_count_add = category_reviews_add.map(lambda row:(row[0],(row[1],1)))
category_review_count_add

PythonRDD[6] at RDD at PythonRDD.scala:53

In [20]:
category_review_count_add.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [22]:
category_review_count_add = category_reviews_add.mapValues(lambda x :(x,1)) # 기본적으로 앞에 값을 키로 생각함. 밸류값만을 다루는 함수 mapValues
category_review_count_add

PythonRDD[7] at RDD at PythonRDD.scala:53

In [23]:
category_review_count_add.collect()

[('중식', (125, 1)),
 ('중식', (235, 1)),
 ('분식', (32, 1)),
 ('분식', (534, 1)),
 ('일식', (223, 1)),
 ('일식', (52, 1)),
 ('일식', (12, 1)),
 ('아시안', (312, 1)),
 ('패스트푸드', (12, 1)),
 ('패스트푸드', (23, 1))]

In [25]:
reduced = category_review_count_add.reduceByKey(lambda a, b : (a[0]+b[0], a[1]+b[1]))
reduced

PythonRDD[12] at RDD at PythonRDD.scala:53

In [26]:
reduced.collect()

[('중식', (360, 2)),
 ('분식', (566, 2)),
 ('일식', (287, 3)),
 ('아시안', (312, 1)),
 ('패스트푸드', (35, 2))]

In [27]:
mean_reduced = reduced.mapValues(lambda x:x[0]/x[1])
mean_reduced

PythonRDD[13] at RDD at PythonRDD.scala:53

In [28]:
mean_reduced.collect()

[('중식', 180.0),
 ('분식', 283.0),
 ('일식', 95.66666666666667),
 ('아시안', 312.0),
 ('패스트푸드', 17.5)]

In [29]:
# shuffling : 데이터가 키에 따라 썪여서 모이는 것. 파티션끼리 데이터 교환이 이루어 진다

In [30]:
sc.stop()